# Ensemble Learning Project - Master MALIA  

Done by : Mohammed Ali EL ADLOUNI


In [15]:
import numpy as np
import pandas as pd
import os
from joblib import Parallel, delayed
from model import *
from utils import *

# Change to the directory where the script is located
os.chdir(os.getcwd())

seed = 55

In [16]:
# dataset 0
train_data_dataset_0   = np.load(r'../dataset/train/dataset_0/kaggle_source_cate_0_train.npy')
train_labels_dataset_0 = np.load(r'../dataset/train/dataset_0/kaggle_source_cate_0_train_label.npy')
test_data_dataset_0    = np.load(r'../dataset/test/dataset_0/kaggle_source_cate_0_test.npy')
test_labels_dataset_0  = np.load(r'../dataset/test/dataset_0/kaggle_source_cate_0_test_label.npy')

# dataset 1
train_data_dataset_1   = np.load(r'../dataset/train/dataset_1/kaggle_source_cate_1_train.npy')
train_labels_dataset_1 = np.load(r'../dataset/train/dataset_1/kaggle_source_cate_1_train_label.npy')
test_data_dataset_1    = np.load(r'../dataset/test/dataset_1/kaggle_source_cate_1_test.npy')
test_labels_dataset_1  = np.load(r'../dataset/test/dataset_1/kaggle_source_cate_1_test_label.npy')

# dataset 2
train_data_dataset_2   = np.load(r'../dataset/train/dataset_2/kaggle_source_cate_2_train.npy')
train_labels_dataset_2 = np.load(r'../dataset/train/dataset_2/kaggle_source_cate_2_train_label.npy')
test_data_dataset_2    = np.load(r'../dataset/test/dataset_2/kaggle_source_cate_2_test.npy')
test_labels_dataset_2  = np.load(r'../dataset/test/dataset_2/kaggle_source_cate_2_test_label.npy')

# dataset 3
train_data_dataset_3   = np.load(r'../dataset/train/dataset_3/kaggle_source_cate_3_train.npy')
train_labels_dataset_3 = np.load(r'../dataset/train/dataset_3/kaggle_source_cate_3_train_label.npy')
test_data_dataset_3    = np.load(r'../dataset/test/dataset_3/kaggle_source_cate_3_test.npy')
test_labels_dataset_3  = np.load(r'../dataset/test/dataset_3/kaggle_source_cate_3_test_label.npy')


In [17]:
for i in range(4):
    print(f"Dataset {i}:")
    print(f"  Train data:  {eval(f'train_data_dataset_{i}').shape}")
    print(f"  Train labels:{eval(f'train_labels_dataset_{i}').shape}")
    print(f"  Test data:   {eval(f'test_data_dataset_{i}').shape}")
    print(f"  Test labels: {eval(f'test_labels_dataset_{i}').shape}")
    print("-" * 40)


Dataset 0:
  Train data:  (41058, 51)
  Train labels:(41058, 2)
  Test data:   (13686, 51)
  Test labels: (13686, 2)
----------------------------------------
Dataset 1:
  Train data:  (41058, 51)
  Train labels:(41058, 2)
  Test data:   (13686, 51)
  Test labels: (13686, 2)
----------------------------------------
Dataset 2:
  Train data:  (41058, 51)
  Train labels:(41058, 2)
  Test data:   (13686, 51)
  Test labels: (13686, 2)
----------------------------------------
Dataset 3:
  Train data:  (41058, 51)
  Train labels:(41058, 2)
  Test data:   (13686, 51)
  Test labels: (13686, 2)
----------------------------------------


In [18]:
for i in range(4):
    var_name = f"train_labels_dataset_{i}"
    calculate_proportions(eval(var_name), var_name)


Proportions for train_labels_dataset_0: {np.int64(0): np.float64(0.8995810804228165), np.int64(1): np.float64(0.10041891957718349)}
Proportions for train_labels_dataset_1: {np.int64(0): np.float64(0.9004335330508062), np.int64(1): np.float64(0.09956646694919383)}
Proportions for train_labels_dataset_2: {np.int64(0): np.float64(0.901675678308734), np.int64(1): np.float64(0.09832432169126601)}
Proportions for train_labels_dataset_3: {np.int64(0): np.float64(0.9018461688343319), np.int64(1): np.float64(0.09815383116566807)}


In [19]:
# Define the datasets with both train and test data
datasets = [
    {
        'X_train': train_data_dataset_0, 
        'y_train': train_labels_dataset_0[:,1], 
        'X_test': test_data_dataset_0, 
        'y_test': test_labels_dataset_0[:,1], 
        'sampling': 'none'  # No sampling
    },
    {
        'X_train': train_data_dataset_1, 
        'y_train': train_labels_dataset_1[:,1], 
        'X_test': test_data_dataset_1, 
        'y_test': test_labels_dataset_1[:,1], 
        'sampling': 'undersampling'  # Undersampling
    },
    {
        'X_train': train_data_dataset_2, 
        'y_train': train_labels_dataset_2[:,1], 
        'X_test': test_data_dataset_2, 
        'y_test': test_labels_dataset_2[:,1], 
        'sampling': 'oversampling'  # Oversampling
    },
    {
        'X_train': train_data_dataset_3, 
        'y_train': train_labels_dataset_3[:,1], 
        'X_test': test_data_dataset_3, 
        'y_test': test_labels_dataset_3[:,1], 
        'sampling': 'cost_sensitive'  # Cost-sensitive learning
    }
]

# Define the models
models = ['Random Forest', 'Bagging', 'Boosting', 'Penalized Logistic Regression', 'Simple Decision Tree', 'XGBoost', 'Stacking']
models = ['Simple Decision Tree', 'XGBoost']

In [20]:
# Parallel execution
# Use joblib's Parallel and delayed to parallelize model runs
results_list = Parallel(n_jobs=-1)(delayed(run_model_on_dataset)(model, dataset, i) 
                                   for i, dataset in enumerate(datasets) 
                                   for model in models)

In [21]:
print("Results List:", results_list)

results_df = pd.DataFrame(results_list)
#results_df.to_csv(r'results/results.csv', index=False)


Results List: [{'dataset': 1, 'sampling': 'none', 'model': 'Simple Decision Tree', 'best_params': {'max_depth': 10}, 'F1 Score': 0.7112616426756986, 'Precision': 0.7932011331444759, 'Recall': 0.6446661550268611, 'Balanced accuracy': np.float64(0.8134903092020359), 'elapsed_time': 1.4602417945861816}, {'dataset': 1, 'sampling': 'none', 'model': 'XGBoost', 'best_params': {'learning_rate': 0.1, 'n_estimators': 100, 'max_depth': 3}, 'F1 Score': 0.7017225747960109, 'Precision': 0.8571428571428571, 'Recall': 0.5940138142747505, 'Balanced accuracy': np.float64(0.7917981532005264), 'elapsed_time': 3.4574952125549316}, {'dataset': 2, 'sampling': 'undersampling', 'model': 'Simple Decision Tree', 'best_params': {'max_depth': 10}, 'F1 Score': 0.543168880455408, 'Precision': 0.39784572619874914, 'Recall': 0.8557548579970105, 'Balanced accuracy': np.float64(0.8577041215802998), 'elapsed_time': 0.38431549072265625}, {'dataset': 2, 'sampling': 'undersampling', 'model': 'XGBoost', 'best_params': {'lear